In [10]:
import pandas as pd

others_list = 'ftp://ftp.nasdaqtrader.com/symboldirectory/otherlisted.txt'
nasdaq_list = 'ftp://ftp.nasdaqtrader.com/symboldirectory/nasdaqlisted.txt'

def symbols_nyse():
    other = pd.read_csv(others_list, sep='|')
    #NYSEのものを取得する
    company_nyse = other[other['Exchange']=='N'][['ACT Symbol', 'Security Name']]
    #ETFはMYSE MKT、NYSE ARCA、MATSがある。
    etf_other = other[other['ETF'] == 'Y'][['ACT Symbol', 'Security Name', 'Exchange']]   
    #indexはリセットする
    company_nyse = company_nyse.reset_index(drop=True)
    etf_other = etf_other.reset_index(drop=True)
    #ACT Symbol -> Symbol
    company_nyse = company_nyse.rename(columns={'ACT Symbol':'Symbol'})
    etf_other = etf_other.rename(columns={'ACT Symbol':'Symbol'})

    return company_nyse, etf_other

def symbols_nasdaq():
    nasdaq = pd.read_csv(nasdaq_list, sep='|')
    #StatusがNormalのものだけを取得する
    nasdaq_normal = nasdaq[nasdaq['Financial Status']=='N']
    #Test issueでないものを選択する
    nasdaq_normal = nasdaq_normal[nasdaq_normal['Test Issue']=='N']
    #ETFかどうかで判別
    company_nasdaq = nasdaq_normal[nasdaq_normal['ETF']=='N'][['Symbol', 'Security Name']]
    etf_nasdaq = nasdaq_normal[nasdaq_normal['ETF']=='Y'][['Symbol', 'Security Name']]
    #indexはリセットする
    company_nasdaq = company_nasdaq.reset_index(drop=True)
    etf_nasdaq = etf_nasdaq.reset_index(drop=True)

    return company_nasdaq, etf_nasdaq

def symbols_all():
    company_nyse, etf_other = symbols_nyse()
    company_nasdaq, etf_nasdaq = symbols_nasdaq()
    #NYSEとNASDAQは区別する
    company_nyse['Market'] = 'NYSE'
    company_nasdaq['Market'] = 'NASDAQ'    
    #NASDAQのETFも区別しておく
    etf_nasdaq['Exchange'] = 'NASDAQ'#etf_otherのcolum名に合わせる

    return (pd.concat([company_nyse, company_nasdaq], ignore_index=True, sort=False),
            pd.concat([etf_other, etf_nasdaq], ignore_index=True, sort=False))



In [11]:
company_nyse, etf_other = symbols_nyse()

In [12]:
company_nyse.to_csv('company_nyse.csv')